# plex

------
current status: **INCOMPLETE**

- goal: install and configure a plex server
- details:
  - single ec2 server running a plex instance: https://www.plex.tv/
  - behind elb
  - ASG
  - s3fs-fuse https://github.com/s3fs-fuse/s3fs-fuse

In [ ]:
from plex_settings import domain_name, pms_rpm_url
from troposphere import Template, Ref, GetAtt, GetAZs, Base64, Join, FindInMap, Select, Output, Parameter, Tags
import troposphere.ec2 as ec2
import troposphere.elasticloadbalancing as elb
import boto3

In [ ]:
resources = set()
t = Template()
t.add_version('2010-09-09')
t.add_description('plex behind an elb and ASG')

# common tag to apply to most resources
domain_name_tags = Tags(Site=domain_name)

# loadbalance across 2 AZs
az1 = Select('0', GetAZs(''))
az2 = Select('1', GetAZs(''))
az3 = Select('2', GetAZs(''))

In [ ]:
plex_vpc = ec2.VPC('plex', CidrBlock='10.0.0.0/16', Tags=domain_name_tags)
resources.add(plex_vpc);

In [ ]:
web_subnet = ec2.Subnet('webserver',
                        AvailabilityZone=az1,
                        CidrBlock='10.0.1.0/24',
                        VpcId=Ref(plex_vpc),
                        Tags=domain_name_tags)
resources.add(web_subnet);

In [ ]:

web_elb = elb.LoadBalancer('webElb',
                           AvailabilityZones=az1,
                           Subnets=Ref(web_subnet),
                           Tags=domain_name_tags)


In [ ]:
file_name = 'plex.json'

def build(filename):
    fname = 'outputs/' + filename
    for r in resources:
        t.add_resource(r)
    
    with open(fname, 'w') as fd:
        fd.write(t.to_json())
    
    !aws cloudformation validate-template --template-body file://{fname} > /dev/null && echo "build ok"

build(file_name)

In [ ]:
client = boto3.client('cloudformation')

In [ ]:
client.create_stack(StackName='plex',
                    TemplateBody=t.to_json(),
                    Parameters=[],
                    Tags=domain_name_tags.to_dict())

In [ ]:
for event in client.describe_stack_events(StackName='plex')['StackEvents']:
    print(event['Timestamp'], event['ResourceStatus'], event['ResourceType'])

In [ ]:
client.delete_stack(StackName='plex')